## 2. Modelo no lineal para la predicción del precio del bitcoin

Acontinuación se plantea un modelo no lineal para predecir el precio del bitcoin mediante el ajuste con mínimos cuadrados.

---

#### Creación del data set
Acontinuación se crea el data set, tomando los precios del bitcoin durante el último año

In [ ]:
using CSV
using DataFrames


#Data set precio del bitcoin durante el último año
dataBitcoin = CSV.read("actualBitcoin.csv",DataFrame);
dataBitcoin.date = dataBitcoin.Date;
dataBitcoin.Price .= replace.(dataBitcoin.Price, "," => "");
dataBitcoin.PriceFloat = parse.(Float64, dataBitcoin.Price);

#### Modelo
Utilizaremos distintos modelos basados en polinomios que van a relacionar los dias con el precio del bitcoin.
- La función a minimizar es $f_0(x)=\|Ax-b\|$ donde $A \in \mathbb{R}^{368\times n+1}$ donde 368 hace referencia al número de datos de entrada, que en este caso son los últimos 368 dias, y $n$ es el grado del polinomio con el que queremos modelar. La $i+1-esima $ columna de $A$ está dada por $ a_1^i,a_2^i,\dots a_{376}^i$, para $i=0,1,2\dots n$, es decir $A$ tendría la siguiente forma:
$$A=\begin{matrix}
1 & a_1 & a_1^2 & \dots & a_1^n\\
1 & a_2 & a_2^2 & \dots & a_2^n \\
\vdots & \vdots  & \vdots& \vdots\\
1 & a_{368} & a_{368}^2 & \dots & a_{368}^n \\
\end{matrix}$$
y x sería un vector que pertenece a $\mathbb{R}^{n+1\times 1}$ y sería el vector $x=(\beta_0,\beta_1,\beta_2,\dots,\beta_n)$ de variables de optimización, por último $b\in \mathbb{R}^{m\times 1}$ y cada $b_i$ corresponde al precio del bitcoin en el día $i$ para $i=1,\dots 368$.De esta manera nuestros modelos son de la forma $\beta_0+\beta_1x+\beta_2x^2+\dots+\beta_nx^n=b$ oara $n\geq 2$.
Acontinuación probaremos el modelo para $n=2,3,4 \ y \ 5$

Gráfico del precio del bitcoin desde el 11 de marzo de 2021 al 13 de marzo de 2022:

In [ ]:
using PlotlyJS, DataFrames, Dates
#convert string databitcoin.Date to type Date
dataBitcoin[!, :Date] = Date.(dataBitcoin.Date, dateformat"uuu d, yyyy");

In [ ]:
plot(dataBitcoin.Date[11:378],dataBitcoin.PriceFloat[11:378],Layout(title="Precio del bitcoin",xaxis_rangeslider_visible=true,layout=(2,2)))

Creación de los modelos

In [ ]:
#Creación de los modelos
m=368
a=dataBitcoin.PriceFloat[11:378]
#start matrix
A=[ones(m,1) Array((1:m))]
b=reverse(dataBitcoin.PriceFloat[11:378])
#arreglo que guarda la prediccion para n=2,3,4,5 
predictions=ones(368,1)
#arreglo que guarda los x de cada modelo 
x_result=[]

pred=0
for n in 2:5
    A=[A Array((1:m)).^n]
    x=A\b
    print(x)
    push!(x_result,x)
    pred=A*x
    predictions=[predictions reverse(pred)]
    #push!(finalPlot,plot(dataBitcoin.Date[11:378],[pred dataBitcoin.Price[11:378]]));
    
end


In [ ]:
print(size(predictions))
print(size(dataBitcoin.Price[11:378]))
#finalplott=[finalPlot[1];finalPlot[2];finalPlot[3];finalPlot[4]]
#relayout!(finalplott, height=1400, width=800,margin=2)
models=plot(dataBitcoin.Date[11:378],[predictions[:,2:5] dataBitcoin.Price[11:378]])
restyle!(models, name=["n=2","n=3","n=4","n=5","precio Bitcoin"])
models

de acuerdo a la gráfica los modelos con polinomios de grado 3 y 4 tenderán a decrecer de manera muy rapida y el de grado 2 no va a tener mucha variación, por lo que haremos las predicciones de los precios de la semana únicamente con el modelo del polinomio de grado  5.

In [ ]:
f5(x)=x_result[4][1]+x*x_result[4][2]+x^2*x_result[4][3]+x^3*x_result[4][4]+x^4*x_result[4][5]+x^5*x_result[4][6]
plot(dataBitcoin.Date[1:10],[reverse(f5.(Array((m+1:378)))) (dataBitcoin.PriceFloat[1:10])])

In [ ]:
#diferencia de la predicción y promedio de diferencia
diferencia=reverse(f5.(Array((m+1:378))))-dataBitcoin.PriceFloat[1:10]
promedio=sum(diferencia)/10

En promedio se tiene que la diferencia entre la predicción y el precio real es de 11581 dolares, con lo cual este modelo no demuestra una buena precisión en su predicción

In [ ]:
plot1=plot(dataBitcoin.Date,dataBitcoin.PriceFloat,Layout(title="Precio del bitcoin",xaxis_rangeslider_visible=true,layout=(2,2)))
plott=[plot1;plot1]
relayout!(plott, height=600, width=600, title_text="Stacked Subplots",margin=2)
plott

In [ ]:
make_plot() = plot(scatter(x=1:4, y=rand(4)))
p = [make_plot(); make_plot(); make_plot()]
relayout!(p, height=600, width=600, title_text="Stacked Subplots")
p